# Smart Shopping Assistant

Author: Xiaohui Chen

## Overview

This notebook demonstrates a **multi-agent shopping assistant** built with the **OpenAI Agents SDK**. The system helps users find, compare, and research products using specialized AI agents that collaborate through **tools** and **handoffs**.

### Architecture

```
                              ┌─────────────────────┐
                              │        USER         │
                              └──────────┬──────────┘
                                         │ ①
                                         ▼
                              ┌─────────────────────┐
                              │  🎯 CONCIERGE AGENT │◄─────────────────────┐
                              │    (Orchestrator)   │                      │
                              └──────────┬──────────┘                      │
                                         │ ②                               │
                           ┌─────────────┴─────────────┐                   │
                           │         HANDOFF           │                   │
                           ▼                           ▼                   │
                ┌─────────────────────┐     ┌─────────────────────┐        │
                │ 🔍 PRODUCT          │     │ ⭐ REVIEW           │        │
                │    SPECIALIST       │     │    ANALYST          │        │ ③
                │                     │     │                     │        │
                │ • Search products   │     │ • Fetch reviews     │        │
                │ • Compare prices    │     │ • Analyze sentiment │        │
                │ • Get specs         │     │ • Summarize pros/   │        │
                │                     │     │   cons              │        │
                └──────────┬──────────┘     └──────────┬──────────┘        │
                           │                           │                   │
                           └───────────────────────────┘                   │
                                         │                                 │
                                         └───── HANDOFF BACK ──────────────┘
                                                     │ ④
                                                     ▼
                                         ┌─────────────────────┐
                                         │       USER          │
                                         │    (Response)       │
                                         └─────────────────────┘
```

**Flow:**
1. User sends a request to the Concierge
2. Concierge analyzes intent and hands off to the appropriate specialist
3. Specialist uses tools to complete the task and hands back to Concierge
4. Concierge synthesizes the response and replies to the user

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Tools** | Functions that agents can call to interact with external APIs (e.g., product search, review fetching) |
| **Handoffs** | Mechanism for one agent to delegate tasks to another specialized agent |
| **Orchestration** | The Concierge agent routes requests to specialists and synthesizes their responses |

### Data Source

All product data is retrieved from the **Real-Time Product Search API** (via RapidAPI), which aggregates results from Google Shopping across multiple retailers including Amazon, Best Buy, Walmart, and more.


## 🎯 Concierge Agent - Low Level Design

### Model Selection

| Attribute | Value |
|-----------|-------|
| **Model** | `gpt-5.2-pro` |
| **Context Window** | 400,000 tokens |
| **Max Output** | 128,000 tokens |
| **Knowledge Cutoff** | August 31, 2025 |
| **Reasoning Effort** | `medium`, `high`, `xhigh` |
| **Input Modalities** | Text, Images |
| **Output Modalities** | Text |

**Supported Features:**

| Feature | Supported |
|---------|-----------|
| Function Calling | ✅ Yes |
| Streaming | ✅ Yes |
| Web Search | ✅ Yes |
| File Search | ✅ Yes |
| MCP (Model Context Protocol) | ✅ Yes |
| Structured Outputs | ❌ No |
| Fine-Tuning | ❌ No |
| Code Interpreter | ❌ No |

**API Endpoint:** `v1/responses`

**Why `gpt-5.2-pro` for Concierge?**

The model is selected for the Concierge Agent due to its superior reasoning capabilities:
- **Complex Reasoning** - Supports `xhigh` reasoning effort for nuanced intent classification
- **Large Context** - 400K tokens allows maintaining full conversation history
- **Function Calling** - Essential for triggering handoffs to specialist agents
- **MCP Support** - Enables integration with external tool servers

### Responsibilities

**Pre-Handoff Phase** (Initial user request):

1. **Receive User Input** - Accept the user's message
2. **Understand Intent** - Determine what the user wants (search, compare, review analysis, etc.)
3. **Decide Routing** - Choose to hand off to a specialist or respond directly
4. **Execute Handoff** - Call the appropriate transfer function (SDK-generated)

**Post-Handoff Phase** (After receiving results from specialists):

1. **Receive Specialist Results** - Accept the handoff back with task results
2. **Synthesize Response** - Format results into a user-friendly response
3. **Offer Next Steps** - Suggest related actions (e.g., "Would you like to see reviews?")
4. **Handle Follow-ups** - Route additional requests or answer questions about results

### Native Handoffs (OpenAI Agents SDK)

The Concierge Agent uses the **native `handoffs` parameter** from the OpenAI Agents SDK. This is the idiomatic way to implement agent-to-agent transfers.

**Bidirectional Handoff Configuration:**

For the Concierge to receive control back from specialists, we use **bidirectional handoffs** - each agent can transfer to the other.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           BIDIRECTIONAL HANDOFFS                            │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│                         ┌─────────────────────┐                             │
│                         │     CONCIERGE       │                             │
│                         │                     │                             │
│                         │ handoffs=[          │                             │
│                         │   product_specialist│                             │
│                         │   review_analyst    │                             │
│                         │ ]                   │                             │
│                         └──────────┬──────────┘                             │
│                                    │                                        │
│              transfer_to_          │           transfer_to_                 │
│              product_specialist    │           review_analyst               │
│                    │               │               │                        │
│                    ▼               │               ▼                        │
│     ┌─────────────────────┐       │       ┌─────────────────────┐          │
│     │ PRODUCT SPECIALIST  │       │       │   REVIEW ANALYST    │          │
│     │                     │       │       │                     │          │
│     │ handoffs=[concierge]│       │       │ handoffs=[concierge]│          │
│     └──────────┬──────────┘       │       └──────────┬──────────┘          │
│                │                  │                  │                      │
│                └──────────────────┼──────────────────┘                      │
│                                   │                                         │
│                    transfer_to_concierge                                    │
│                                   │                                         │
│                                   ▼                                         │
│                         ┌─────────────────────┐                             │
│                         │     CONCIERGE       │                             │
│                         │   (receives results)│                             │
│                         └─────────────────────┘                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

**Transfer Tools Created by SDK:**

| Agent | Has Transfer Tool | Can Transfer To |
|-------|-------------------|-----------------|
| **Concierge** | `transfer_to_product_specialist` | Product Specialist |
| **Concierge** | `transfer_to_review_analyst` | Review Analyst |
| **Product Specialist** | `transfer_to_concierge` | Concierge |
| **Review Analyst** | `transfer_to_concierge` | Concierge |

**Handoff Descriptions:**

| From | To | Description |
|------|-----|-------------|
| Concierge | Product Specialist | Search for products, compare prices, get specifications |
| Concierge | Review Analyst | Analyze product reviews, summarize customer feedback |
| Product Specialist | Concierge | Return results after completing product search/details task |
| Review Analyst | Concierge | Return results after completing review analysis task |

**Benefits of Bidirectional Handoffs:**
- ✅ Full orchestration control - Concierge can synthesize final responses
- ✅ Supports multi-step workflows (e.g., search → review → recommendation)
- ✅ Conversation context preserved throughout the cycle
- ✅ Specialists focus on tasks, Concierge handles user communication

### Routing Logic

| User Request Type | Action | Transfer Tool |
|-------------------|--------|---------------|
| Product search ("find me a laptop") | Hand off | `transfer_to_product_specialist` |
| Price comparison ("best deal") | Hand off | `transfer_to_product_specialist` |
| Specifications ("what's the processor") | Hand off | `transfer_to_product_specialist` |
| Review analysis ("what do people say") | Hand off | `transfer_to_review_analyst` |
| Sentiment questions ("any complaints") | Hand off | `transfer_to_review_analyst` |
| General questions ("what can you do") | Respond directly | None |
| Ambiguous requests ("I need something") | Ask clarification | None |

### System Prompt

```
You are the Concierge Agent for a Smart Shopping Assistant. Your role is to understand 
user requests, route them to specialist agents, and synthesize their results for the user.

## Your Responsibilities

1. GREET users and understand their shopping needs
2. ROUTE requests to the appropriate specialist by calling the transfer function
3. RECEIVE results back from specialists and present them to the user
4. OFFER next steps and handle follow-up requests
5. HANDLE general questions about your capabilities directly
6. ASK for clarification when requests are too vague

## Available Specialists

You have access to two specialist agents via transfer functions:

### Product Specialist (transfer_to_product_specialist)
Transfer to this agent when user wants to:
- Search for products ("find me a laptop", "show me TVs under $500")
- Compare prices across stores ("where's the best deal", "cheapest option")
- Get product specifications ("what's the processor", "specs for MacBook")
- Filter or sort products ("highest rated gaming laptops")

The Product Specialist has access to product search and details tools.

### Review Analyst (transfer_to_review_analyst)
Transfer to this agent when user wants to:
- Read or understand reviews ("what do people say about...")
- Analyze product quality ("is this reliable", "any complaints")
- Understand pros and cons ("what's good/bad about...")
- See rating breakdowns ("how many 5-star reviews")

The Review Analyst has access to review fetching and sentiment analysis tools.

## When NOT to Transfer

Handle these yourself (do not transfer):
- General questions: "What can you help me with?", "How does this work?"
- Greetings: "Hi", "Hello"
- Vague requests that need clarification first
- Follow-up questions that don't need specialist tools

## When Receiving Results from Specialists

After a specialist completes their task and transfers back to you:

1. PRESENT the results in a clear, user-friendly format
2. HIGHLIGHT key information (best prices, top-rated products, main pros/cons)
3. OFFER relevant next steps:
   - After product search: "Would you like to see reviews for any of these?"
   - After review analysis: "Would you like to search for alternatives?"
   - After price comparison: "Would you like more details on the cheapest option?"
4. BE READY to route follow-up requests to the appropriate specialist

## Clarification Guidelines

If a request is too vague to route, ask for clarification. Examples:
- "I need something" → Ask what type of product and budget
- "Help me shop" → Ask what they're looking for
- "What's good?" → Ask what category they're interested in

Always be friendly and helpful when asking for more information.

## Examples

### Initial Routing

User: "Find me a gaming laptop under $1500"
→ Transfer to Product Specialist (product search request with clear criteria)

User: "What do reviews say about the MacBook Air battery?"
→ Transfer to Review Analyst (review analysis with specific focus area)

User: "Where can I get the best price on AirPods Pro?"
→ Transfer to Product Specialist (price comparison request)

User: "Is the Samsung S24 reliable?"
→ Transfer to Review Analyst (quality/reliability question)

User: "I need a new phone"
→ Respond directly: Ask clarifying questions about budget and preferences

### After Receiving Results

[After Product Specialist returns with laptop search results]
→ Present the top options with prices and key specs
→ Offer: "Would you like me to check the reviews for any of these options?"

[After Review Analyst returns with MacBook Air review summary]
→ Present the pros, cons, and overall sentiment
→ Offer: "Would you like to compare prices or see alternatives?"
```

### Handoff Flow Diagram (Complete Cycle)

```
┌─────────────────────────────────────────────────────────────────┐
│                        USER MESSAGE                              │
│         "Find me a gaming laptop under $1500"                   │
└─────────────────────────────────────────────────────────────────┘
                               │
                               │ ① User Request
                               ▼
                    ┌─────────────────────┐
                    │   CONCIERGE AGENT   │
                    │   (gpt-5.2-pro)     │
                    │                     │
                    │   Handoffs:         │
                    │   • product_        │
                    │     specialist      │
                    │   • review_analyst  │
                    └──────────┬──────────┘
                               │
                               │ ② Decides to transfer
                               │   (product search request)
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   transfer_to_      │
                    │   product_specialist│
                    └──────────┬──────────┘
                               │
                               │ ③ SDK transfers control
                               ▼
                    ┌─────────────────────┐
                    │ PRODUCT SPECIALIST  │
                    │                     │
                    │   Tools:            │
                    │   • search_products │
                    │   • get_product_    │
                    │     details         │
                    │                     │
                    │   Handoffs:         │
                    │   • concierge       │
                    └──────────┬──────────┘
                               │
                               │ ④ Executes tools, gets results
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   search_products(  │
                    │     "gaming laptop",│
                    │     max_price=1500  │
                    │   )                 │
                    │                     │
                    │   Returns: 5 laptops│
                    │   with prices       │
                    └──────────┬──────────┘
                               │
                               │ ⑤ Task complete, transfer back
                               ▼
                    ┌─────────────────────┐
                    │   CALLS:            │
                    │   transfer_to_      │
                    │   concierge         │
                    │                     │
                    │   (with results in  │
                    │    conversation     │
                    │    context)         │
                    └──────────┬──────────┘
                               │
                               │ ⑥ SDK transfers control back
                               ▼
                    ┌─────────────────────┐
                    │   CONCIERGE AGENT   │
                    │   (receives results)│
                    │                     │
                    │   • Synthesizes     │
                    │     response        │
                    │   • Formats for     │
                    │     user            │
                    │   • Offers next     │
                    │     steps           │
                    └──────────┬──────────┘
                               │
                               │ ⑦ Final response
                               ▼
                    ┌─────────────────────┐
                    │   USER RESPONSE     │
                    │                     │
                    │   "Here are 5       │
                    │   gaming laptops    │
                    │   under $1500:      │
                    │   1. ASUS ROG...    │
                    │   2. MSI Stealth... │
                    │   ...               │
                    │                     │
                    │   Would you like to │
                    │   see reviews for   │
                    │   any of these?"    │
                    └─────────────────────┘
```


## 🔍 Product Specialist Agent - Low Level Design

### Model Selection

| Attribute | Value |
|-----------|-------|
| **Model** | `gpt-5.2` |
| **Provider** | OpenAI |
| **Context Window** | 128,000 tokens |
| **Max Output** | 16,384 tokens |
| **Knowledge Cutoff** | August 31, 2025 |
| **Input Modalities** | Text, Images |
| **Output Modalities** | Text |

**Supported Features:**

| Feature | Supported |
|---------|-----------|
| Function Calling | ✅ Yes |
| Streaming | ✅ Yes |
| Structured Outputs | ✅ Yes |
| Web Search | ✅ Yes |
| File Search | ✅ Yes |
| Fine-Tuning | ❌ No |
| Code Interpreter | ❌ No |

**API Endpoint:** `v1/responses`

**Why `gpt-5.2` for Product Specialist?**

- **Efficient Tool Use** - Reliable function calling for API interactions
- **Cost-Effective** - Standard model pricing vs. pro tier for orchestration
- **Fast Response** - Lower latency compared to reasoning models
- **Native SDK Support** - Enables bidirectional handoffs with Concierge (gpt-5.2-pro)
- **Sufficient Context** - 128K tokens handles product search results well

### Responsibilities

The Product Specialist Agent handles all product-related tasks:

1. **Search Products** - Find products matching user criteria (keywords, price, features)
2. **Compare Prices** - Show prices from multiple retailers (via offers in search results)
3. **Get Specifications** - Retrieve detailed product specs when requested
4. **Filter & Sort** - Apply sorting and filtering based on user preferences
5. **Hand Back to Concierge** - Transfer control back with results for final presentation

### Tools

The Product Specialist has access to **2 tools** that interact with the **Real-Time Product Search API**.

#### Tool 1: `search_products`

| Property | Value |
|----------|-------|
| **Name** | `search_products` |
| **Description** | Search for products across multiple retailers. Returns products with prices from different stores. |
| **API Endpoint** | `GET https://real-time-product-search.p.rapidapi.com/search` |

**Parameters:**

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `query` | string | ✅ | Search keywords (e.g., "gaming laptop 16GB RAM") |
| `min_price` | number | ❌ | Minimum price filter in USD |
| `max_price` | number | ❌ | Maximum price filter in USD |
| `sort_by` | string | ❌ | Sort order: `BEST_MATCH`, `LOWEST_PRICE`, `HIGHEST_PRICE`, `TOP_RATED` |
| `condition` | string | ❌ | Product condition: `NEW`, `USED`, `REFURBISHED` (default: `NEW`) |
| `country` | string | ❌ | Country code (default: `us`) |
| `limit` | integer | ❌ | Number of results (default: 10) |

**Returns:**

| Field | Type | Description |
|-------|------|-------------|
| `products` | array | List of matching products |
| `products[].product_id` | string | Unique ID (use for get_product_details) |
| `products[].title` | string | Product title |
| `products[].description` | string | Short description |
| `products[].price` | string | Formatted price (e.g., "$1,299.99") |
| `products[].extracted_price` | number | Numeric price for comparison |
| `products[].rating` | number | Average rating (1-5) |
| `products[].reviews` | integer | Number of reviews |
| `products[].offers` | array | Prices from different stores |
| `products[].offers[].store` | string | Retailer name |
| `products[].offers[].price` | string | Price at this store |
| `products[].images` | array | Product image URLs |

**API Details:**

```
Host: real-time-product-search.p.rapidapi.com
Endpoint: GET /search
Headers:
  - X-RapidAPI-Key: {RAPIDAPI_KEY}
  - X-RapidAPI-Host: real-time-product-search.p.rapidapi.com
```

---

#### Tool 2: `get_product_details`

| Property | Value |
|----------|-------|
| **Name** | `get_product_details` |
| **Description** | Get comprehensive details and specifications for a specific product. |
| **API Endpoint** | `GET https://real-time-product-search.p.rapidapi.com/product` |

**Parameters:**

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `product_id` | string | ✅ | Product ID from search results |
| `country` | string | ❌ | Country code for localized pricing (default: `us`) |

**Returns:**

| Field | Type | Description |
|-------|------|-------------|
| `product_id` | string | Product identifier |
| `title` | string | Full product title |
| `description` | string | Complete product description |
| `specifications` | object | Detailed specs (processor, RAM, storage, etc.) |
| `images` | array | All product images |
| `rating` | number | Average rating |
| `reviews_count` | integer | Total reviews |
| `offers` | array | All available prices from retailers |
| `offers[].store` | string | Retailer name |
| `offers[].price` | string | Price |
| `offers[].extracted_price` | number | Numeric price |
| `offers[].delivery` | string | Delivery information |
| `offers[].availability` | string | Stock status |

**API Details:**

```
Host: real-time-product-search.p.rapidapi.com
Endpoint: GET /product
Headers:
  - X-RapidAPI-Key: {RAPIDAPI_KEY}
  - X-RapidAPI-Host: real-time-product-search.p.rapidapi.com
Query Params:
  - product_id: {product_id}
  - country: {country}
```

### Handoff Configuration

| Direction | Transfer Tool | Description |
|-----------|---------------|-------------|
| **To Concierge** | `transfer_to_concierge` | Return results after completing product task |

The Product Specialist has only one handoff target (Concierge) since it should always return results to the orchestrator rather than directly to the user.

### System Prompt

```
You are the Product Specialist for a Smart Shopping Assistant. You help users find and 
compare products using your search and details tools.

## Your Role

You receive requests from the Concierge Agent to find products, compare prices, or get 
specifications. After completing your task, you MUST transfer back to the Concierge 
with your findings.

## Your Tools

### search_products
Use this tool to find products matching user criteria.
- Always include relevant keywords from the user's request in the query
- Apply price filters when the user specifies a budget
- Use sort_by to match user preferences:
  - "cheapest" → sort_by: "LOWEST_PRICE"
  - "best rated" → sort_by: "TOP_RATED"
  - "most relevant" → sort_by: "BEST_MATCH"
- Default to NEW condition unless user asks for used/refurbished

### get_product_details
Use this tool when you need:
- Full technical specifications
- Complete price comparison across all stores
- Detailed product information

Only use this after search_products when you have a product_id, or when the user 
asks about a specific product's specs.

## Workflow

1. RECEIVE the request from Concierge (user wants products, prices, or specs)
2. ANALYZE what tools you need:
   - Product search → use search_products
   - Need specs for specific product → use get_product_details
   - Price comparison → use search_products (offers are included)
3. EXECUTE the appropriate tool(s)
4. SUMMARIZE the results clearly:
   - For search: List top products with key info (title, price, rating, stores)
   - For specs: Present specifications in an organized format
   - For prices: Show the price comparison across stores
5. TRANSFER back to Concierge using transfer_to_concierge

## Guidelines

### Search Queries
- Extract the main product type: "gaming laptop with RTX 4060" → query: "gaming laptop RTX 4060"
- Include key features mentioned: "16GB RAM", "15 inch screen", etc.
- Don't over-specify: let the API find relevant matches

### Price Handling
- "under $1500" → max_price: 1500
- "at least $500" → min_price: 500
- "$500-$1000" → min_price: 500, max_price: 1000
- "around $800" → min_price: 700, max_price: 900

### Presenting Results
- Always mention the number of results found
- Highlight the best value option (good rating + competitive price)
- Note price differences between stores when significant
- Include ratings and review counts for credibility

### When to Get Details
- User asks "what's the processor/RAM/specs"
- User wants to compare two specific products
- User needs more info before deciding

## Important

- ALWAYS transfer back to Concierge after completing your task
- Do NOT respond directly to the user - let Concierge handle user communication
- Include enough context in your summary for Concierge to present to the user
```

### Execution Flow

```
┌─────────────────────────────────────────────────────────────────┐
│          PRODUCT SPECIALIST RECEIVES HANDOFF                     │
│          (from Concierge with user's product request)           │
└─────────────────────────────────────────────────────────────────┘
                               │
                               ▼
                    ┌─────────────────────┐
                    │   ANALYZE REQUEST   │
                    │                     │
                    │   What does user    │
                    │   need?             │
                    │   • Product search? │
                    │   • Specifications? │
                    │   • Price compare?  │
                    └──────────┬──────────┘
                               │
              ┌────────────────┼────────────────┐
              ▼                ▼                ▼
      ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
      │   SEARCH    │  │   DETAILS   │  │   BOTH      │
      │             │  │             │  │             │
      │ search_     │  │ get_product_│  │ search_     │
      │ products()  │  │ details()   │  │ products()  │
      │             │  │             │  │     +       │
      │             │  │             │  │ get_product_│
      │             │  │             │  │ details()   │
      └──────┬──────┘  └──────┬──────┘  └──────┬──────┘
              │                │                │
              └────────────────┼────────────────┘
                               │
                               ▼
                    ┌─────────────────────┐
                    │   PROCESS RESULTS   │
                    │                     │
                    │   • Format products │
                    │   • Highlight best  │
                    │     options         │
                    │   • Note price      │
                    │     differences     │
                    └──────────┬──────────┘
                               │
                               ▼
                    ┌─────────────────────┐
                    │   TRANSFER BACK     │
                    │                     │
                    │   transfer_to_      │
                    │   concierge()       │
                    │                     │
                    │   (with summarized  │
                    │    results in       │
                    │    conversation)    │
                    └─────────────────────┘
```

### Example Tool Calls

**Example 1: Product Search**

User request (via Concierge): "Find me a gaming laptop under $1500"

```
Tool: search_products
Arguments: {
  "query": "gaming laptop",
  "max_price": 1500,
  "sort_by": "RELEVANCE",
  "condition": "NEW",
  "limit": 10
}
```

**Example 2: Price Comparison**

User request (via Concierge): "What's the best deal on AirPods Pro?"

```
Tool: search_products
Arguments: {
  "query": "AirPods Pro",
  "sort_by": "LOWEST_PRICE",
  "limit": 5
}
```

**Example 3: Get Specifications**

User request (via Concierge): "What's the processor in the ASUS ROG Strix?"

```
Tool: get_product_details
Arguments: {
  "product_id": "12345678901234567890",
  "country": "us"
}
```


## ⭐ Review Analyst Agent - Low Level Design

### Model Selection

| Attribute | Value |
|-----------|-------|
| **Model** | `gpt-5.2` |
| **Provider** | OpenAI |
| **Context Window** | 128,000 tokens |
| **Max Output** | 16,384 tokens |
| **Knowledge Cutoff** | August 31, 2025 |
| **Input Modalities** | Text, Images |
| **Output Modalities** | Text |

**Supported Features:**

| Feature | Supported |
|---------|-----------|
| Function Calling | ✅ Yes |
| Streaming | ✅ Yes |
| Structured Outputs | ✅ Yes |
| Web Search | ✅ Yes |
| File Search | ✅ Yes |
| Fine-Tuning | ❌ No |
| Code Interpreter | ❌ No |

**API Endpoint:** `v1/responses`

**Why `gpt-5.2` for Review Analyst?**

- **Nuanced Understanding** - Excels at understanding sentiment, tone, and context in reviews
- **Summarization** - Can distill large amounts of review text into actionable insights
- **Pattern Recognition** - Identifies common themes, complaints, and praise across reviews
- **Native SDK Support** - Enables bidirectional handoffs with Concierge (gpt-5.2-pro)
- **Cost-Effective** - Standard model pricing suitable for review analysis tasks

### Responsibilities

The Review Analyst Agent handles all review-related tasks:

1. **Fetch Reviews** - Retrieve customer reviews for specific products
2. **Analyze Sentiment** - Determine overall sentiment and identify patterns
3. **Summarize Feedback** - Extract key pros, cons, and themes from reviews
4. **Rating Analysis** - Break down ratings (5-star, 4-star distribution, etc.)
5. **Hand Back to Concierge** - Transfer control back with analysis results

### Tools

The Review Analyst has access to **2 tools** - one interacts with the **Real-Time Product Search API**, and one performs local sentiment analysis.

#### Tool 1: `get_product_reviews`

| Property | Value |
|----------|-------|
| **Name** | `get_product_reviews` |
| **Description** | Fetch customer reviews and ratings for a specific product. |
| **API Endpoint** | `GET https://real-time-product-search.p.rapidapi.com/product-reviews-v2` |

**Parameters:**

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `product_id` | string | ✅ | Product ID from search_products (long string like `catalogid:123...`) |
| `country` | string | ❌ | Country code (default: `us`) |
| `language` | string | ❌ | Language code (default: `en`) |
| `limit` | integer | ❌ | Number of reviews to fetch (default: 10) |
| `sort_by` | string | ❌ | Sort order: `MOST_RELEVANT` or `DATE` (default: `MOST_RELEVANT`) |

> ⚠️ **IMPORTANT**: The `product_id` must be the exact ID returned from `search_products`. Do NOT make up product IDs like "sony_wh_1000xm4" - these will return empty results.

**Returns:**

| Field | Type | Description |
|-------|------|-------------|
| `product_id` | string | Product identifier |
| `average_rating` | number | Overall average rating (1-5) |
| `total_reviews` | integer | Total number of reviews |
| `rating_breakdown` | object | Distribution of ratings |
| `rating_breakdown.5_star` | integer | Count of 5-star reviews |
| `rating_breakdown.4_star` | integer | Count of 4-star reviews |
| `rating_breakdown.3_star` | integer | Count of 3-star reviews |
| `rating_breakdown.2_star` | integer | Count of 2-star reviews |
| `rating_breakdown.1_star` | integer | Count of 1-star reviews |
| `reviews` | array | List of individual reviews |
| `reviews[].author` | string | Reviewer name |
| `reviews[].rating` | integer | Rating given (1-5) |
| `reviews[].title` | string | Review title/headline |
| `reviews[].content` | string | Full review text |
| `reviews[].date` | string | Review date |
| `reviews[].source` | string | Review source (e.g., "Amazon", "Best Buy") |
| `reviews[].verified_purchase` | boolean | Whether purchase was verified |
| `reviews[].helpful_votes` | integer | Number of "helpful" votes |

**API Details:**

```
Host: real-time-product-search.p.rapidapi.com
Endpoint: GET /product-reviews-v2
Headers:
  - X-RapidAPI-Key: {RAPIDAPI_KEY}
  - X-RapidAPI-Host: real-time-product-search.p.rapidapi.com
Query Params:
  - product_id: {product_id}  (MUST be from search_products results)
  - country: {country}
  - language: {language}
  - limit: {limit}
  - sort_by: MOST_RELEVANT | DATE
```

---

#### Tool 2: `analyze_sentiment`

| Property | Value |
|----------|-------|
| **Name** | `analyze_sentiment` |
| **Description** | Analyze review text to extract sentiment, themes, pros, and cons. |
| **Type** | Local processing (no external API) |

**Parameters:**

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `reviews` | array | ✅ | Array of review objects from get_product_reviews |
| `focus_area` | string | ❌ | Specific aspect to analyze (e.g., "battery", "durability") |

**Returns:**

| Field | Type | Description |
|-------|------|-------------|
| `overall_sentiment` | string | `positive`, `negative`, `mixed`, or `neutral` |
| `sentiment_score` | number | Score from -1.0 (negative) to 1.0 (positive) |
| `total_analyzed` | integer | Number of reviews analyzed |
| `pros` | array | List of commonly mentioned positive aspects |
| `cons` | array | List of commonly mentioned negative aspects |
| `common_themes` | array | Frequently mentioned topics |
| `notable_reviews` | array | Standout helpful or detailed reviews |
| `red_flags` | array | Serious concerns mentioned by multiple reviewers |
| `recommendation` | string | Brief recommendation based on analysis |

**Processing Logic:**

```
┌─────────────────────────────────────────────────────────────────┐
│                    SENTIMENT ANALYSIS FLOW                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Categorize reviews by rating                                │
│     • Positive: 4-5 stars                                       │
│     • Neutral: 3 stars                                          │
│     • Negative: 1-2 stars                                       │
│                                                                 │
│  2. Extract keywords from each category                         │
│     • Positive keywords: "great", "excellent", "love", etc.     │
│     • Negative keywords: "poor", "broken", "disappointed", etc. │
│                                                                 │
│  3. Identify common themes                                      │
│     • Group by product aspects (battery, screen, build, etc.)   │
│     • Count frequency of mentions                               │
│                                                                 │
│  4. Calculate sentiment score                                   │
│     • Weighted average based on ratings and review length       │
│                                                                 │
│  5. Generate summary                                            │
│     • Top 3-5 pros                                              │
│     • Top 3-5 cons                                              │
│     • Overall recommendation                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Handoff Configuration

| Direction | Transfer Tool | Description |
|-----------|---------------|-------------|
| **To Concierge** | `transfer_to_concierge` | Return analysis results after completing review task |

The Review Analyst has only one handoff target (Concierge) since it should always return results to the orchestrator rather than directly to the user.

### System Prompt

```
You are the Review Analyst for a Smart Shopping Assistant. You help users understand 
product reviews and make informed purchasing decisions.

## Your Role

You receive requests from the Concierge Agent to analyze product reviews. After completing 
your analysis, you MUST transfer back to the Concierge with your findings.

## Your Tools

### get_product_reviews
Use this tool to fetch customer reviews for a product.
- Requires a product_id (usually provided in the conversation context)
- Returns individual reviews with ratings, text, and metadata
- Also returns the overall rating breakdown

### analyze_sentiment
Use this tool after fetching reviews to extract insights.
- Identifies common pros and cons
- Calculates overall sentiment
- Highlights notable reviews and red flags
- Can focus on specific aspects (e.g., "battery life", "build quality")

## Workflow

1. RECEIVE the request from Concierge (user wants review analysis)
2. IDENTIFY the product:
   - Look for product_id in conversation context
   - If only product name is given, note this for Concierge to get ID first
3. FETCH reviews using get_product_reviews
4. ANALYZE the reviews using analyze_sentiment:
   - If user asked about specific aspect, use focus_area parameter
   - Otherwise, provide general analysis
5. SUMMARIZE your findings:
   - Overall rating and sentiment
   - Key pros (what people love)
   - Key cons (common complaints)
   - Notable patterns or red flags
   - Your recommendation
6. TRANSFER back to Concierge using transfer_to_concierge

## Guidelines

### When Analyzing Reviews

- **Prioritize verified purchases** - These are more trustworthy
- **Consider review recency** - Recent reviews reflect current product quality
- **Weight by helpfulness** - Reviews with more helpful votes are more valuable
- **Look for patterns** - Single complaints may be outliers; repeated issues are significant

### Focus Areas

When user asks about specific aspects, focus your analysis:
- "battery life" → Look for mentions of battery, charge time, usage duration
- "build quality" → Look for durability, materials, construction comments
- "value for money" → Compare price mentions to satisfaction
- "customer service" → Look for support, warranty, return experience

### Sentiment Classification

- **Positive** (0.5 to 1.0): Mostly praise, minor complaints
- **Mixed** (0.0 to 0.5): Balanced pros and cons
- **Neutral** (-0.2 to 0.2): Neither strong praise nor complaints
- **Negative** (-1.0 to -0.2): Significant complaints, issues

### Red Flags to Identify

- Safety issues (overheating, defects)
- Quality control problems (DOA, early failures)
- Misleading descriptions
- Pattern of recent negative reviews (quality decline)

### Presenting Analysis

Structure your summary clearly:
1. **Overview**: "Based on X reviews, this product has a Y rating"
2. **Pros**: Top 3-5 things people love
3. **Cons**: Top 3-5 common complaints
4. **Red Flags**: Any serious concerns (if applicable)
5. **Verdict**: Brief recommendation

## Important

- ALWAYS transfer back to Concierge after completing your analysis
- Do NOT respond directly to the user - let Concierge handle user communication
- If product_id is missing, inform Concierge so they can get it via Product Specialist
- Include enough context in your summary for Concierge to present to the user
```

### Execution Flow

```
┌─────────────────────────────────────────────────────────────────┐
│           REVIEW ANALYST RECEIVES HANDOFF                        │
│           (from Concierge with review analysis request)          │
└─────────────────────────────────────────────────────────────────┘
                               │
                               ▼
                    ┌─────────────────────┐
                    │   CHECK FOR         │
                    │   PRODUCT_ID        │
                    │                     │
                    │   Is product_id     │
                    │   available?        │
                    └──────────┬──────────┘
                               │
              ┌────────────────┴────────────────┐
              ▼                                 ▼
       ┌─────────────┐                  ┌─────────────┐
       │    YES      │                  │     NO      │
       │             │                  │             │
       │ Proceed to  │                  │ Transfer    │
       │ fetch       │                  │ back to     │
       │ reviews     │                  │ Concierge   │
       └──────┬──────┘                  │ (need ID)   │
              │                         └─────────────┘
              ▼
   ┌─────────────────────┐
   │   FETCH REVIEWS     │
   │                     │
   │   get_product_      │
   │   reviews(          │
   │     product_id,     │
   │     limit=20        │
   │   )                 │
   └──────────┬──────────┘
              │
              ▼
   ┌─────────────────────┐
   │   ANALYZE REVIEWS   │
   │                     │
   │   analyze_sentiment(│
   │     reviews,        │
   │     focus_area      │
   │   )                 │
   └──────────┬──────────┘
              │
              ▼
   ┌─────────────────────┐
   │   COMPILE SUMMARY   │
   │                     │
   │   • Overall rating  │
   │   • Sentiment score │
   │   • Top pros        │
   │   • Top cons        │
   │   • Red flags       │
   │   • Recommendation  │
   └──────────┬──────────┘
              │
              ▼
   ┌─────────────────────┐
   │   TRANSFER BACK     │
   │                     │
   │   transfer_to_      │
   │   concierge()       │
   │                     │
   │   (with analysis    │
   │    results)         │
   └─────────────────────┘
```

### Example Tool Calls

**Example 1: General Review Analysis**

User request (via Concierge): "What do people say about the MacBook Air?"

```
Tool: get_product_reviews
Arguments: {
  "product_id": "12345678901234567890",
  "country": "us",
  "limit": 20
}

↓ Then ↓

Tool: analyze_sentiment
Arguments: {
  "reviews": [<reviews from previous call>]
}
```

**Example 2: Focused Analysis (Battery)**

User request (via Concierge): "How's the battery life on the Samsung S24?"

```
Tool: get_product_reviews
Arguments: {
  "product_id": "98765432109876543210",
  "country": "us",
  "limit": 20
}

↓ Then ↓

Tool: analyze_sentiment
Arguments: {
  "reviews": [<reviews from previous call>],
  "focus_area": "battery"
}
```

**Example 3: Reliability Check**

User request (via Concierge): "Is the Sony WH-1000XM5 reliable? Any complaints?"

```
Tool: get_product_reviews
Arguments: {
  "product_id": "55555555555555555555",
  "country": "us",
  "limit": 30
}

↓ Then ↓

Tool: analyze_sentiment
Arguments: {
  "reviews": [<reviews from previous call>],
  "focus_area": "durability"
}
```

### Sample Analysis Output

```
┌─────────────────────────────────────────────────────────────────┐
│                    REVIEW ANALYSIS SUMMARY                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Product: MacBook Air M3                                        │
│  Reviews Analyzed: 234                                          │
│  Average Rating: 4.6/5                                          │
│  Overall Sentiment: POSITIVE (score: 0.72)                      │
│                                                                 │
│  ✅ PROS:                                                       │
│  • Exceptional battery life (18+ hours mentioned frequently)    │
│  • Silent operation (fanless design praised)                    │
│  • Lightweight and portable                                     │
│  • Fast performance for daily tasks                             │
│  • Beautiful display quality                                    │
│                                                                 │
│  ❌ CONS:                                                       │
│  • Limited to 8GB RAM in base model                             │
│  • Only 2 USB-C ports                                           │
│  • No Face ID                                                   │
│  • Price premium over competitors                               │
│                                                                 │
│  ⚠️ RED FLAGS:                                                  │
│  • None significant                                             │
│                                                                 │
│  📊 RATING BREAKDOWN:                                           │
│  ★★★★★  5-star: 156 (67%)                                      │
│  ★★★★☆  4-star: 52 (22%)                                       │
│  ★★★☆☆  3-star: 18 (8%)                                        │
│  ★★☆☆☆  2-star: 5 (2%)                                         │
│  ★☆☆☆☆  1-star: 3 (1%)                                         │
│                                                                 │
│  💡 RECOMMENDATION:                                             │
│  Highly recommended for users prioritizing portability and      │
│  battery life. Consider 16GB RAM upgrade for power users.       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```


In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file in repo root
load_dotenv("../../.env")

# Verify required API keys are loaded
# Note: Only OPENAI_API_KEY and RAPID_API_KEY are required
# All agents use OpenAI models for native SDK handoffs
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RAPID_API_KEY = os.getenv("RAPID_API_KEY")

missing_keys = []
if not OPENAI_API_KEY:
    missing_keys.append("OPENAI_API_KEY")
if not RAPID_API_KEY:
    missing_keys.append("RAPID_API_KEY")

if missing_keys:
    print(f"❌ Missing required API keys: {', '.join(missing_keys)}")
else:
    print("✅ All required API keys loaded (OPENAI_API_KEY, RAPID_API_KEY)")


✅ All required API keys loaded (OPENAI_API_KEY, RAPID_API_KEY)


In [2]:
import httpx
from typing import Optional
from agents import function_tool

# RapidAPI configuration
RAPIDAPI_HOST = "real-time-product-search.p.rapidapi.com"
RAPIDAPI_BASE_URL = f"https://{RAPIDAPI_HOST}"

def get_rapidapi_headers() -> dict:
    """Get headers for RapidAPI requests."""
    return {
        "x-rapidapi-key": RAPID_API_KEY,
        "x-rapidapi-host": RAPIDAPI_HOST
    }


# Valid sort_by options for the Real-Time Product Search API v2
VALID_SORT_OPTIONS = {"BEST_MATCH", "LOWEST_PRICE", "HIGHEST_PRICE", "TOP_RATED"}


@function_tool
async def search_products(
    query: str,
    min_price: Optional[float] = None,
    max_price: Optional[float] = None,
    sort_by: str = "BEST_MATCH",
    product_condition: str = "ANY",
    country: str = "us",
    language: str = "en",
    limit: int = 10,
    page: int = 1
) -> dict:
    """
    Search for products across multiple retailers using Real-Time Product Search API v2.
    
    Args:
        query: Search keywords (e.g., "gaming laptop 16GB RAM")
        min_price: Minimum price filter in USD
        max_price: Maximum price filter in USD
        sort_by: Sort order - BEST_MATCH, LOWEST_PRICE, HIGHEST_PRICE, or TOP_RATED
        product_condition: Product condition - ANY, NEW, USED, or REFURBISHED
        country: Country code for localized results (default: us)
        language: Language code (default: en)
        limit: Number of results to return (default: 10)
        page: Page number for pagination (default: 1)
    
    Returns:
        Dictionary containing list of products with prices, ratings, and offers from different stores
    """
    # Validate and normalize sort_by parameter
    # Map common alternatives to valid API options
    sort_mapping = {
        "REVIEW_SCORE": "TOP_RATED",
        "RATING": "TOP_RATED",
        "RELEVANCE": "BEST_MATCH",
    }
    normalized_sort = sort_mapping.get(sort_by.upper(), sort_by.upper())
    if normalized_sort not in VALID_SORT_OPTIONS:
        normalized_sort = "BEST_MATCH"  # Default fallback
    
    params = {
        "q": query,
        "country": country,
        "language": language,
        "limit": str(limit),
        "page": str(page),
        "sort_by": normalized_sort,
        "product_condition": product_condition
    }
    
    if min_price is not None:
        params["min_price"] = str(int(min_price))  # Convert to int string
    if max_price is not None:
        params["max_price"] = str(int(max_price))  # Convert to int string
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/search-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()


@function_tool
async def get_product_details(
    product_id: str,
    country: str = "us",
    language: str = "en"
) -> dict:
    """
    Get comprehensive details and specifications for a specific product.
    
    Args:
        product_id: Product ID from search results
        country: Country code for localized pricing (default: us)
        language: Language code (default: en)
    
    Returns:
        Dictionary containing full product details including specifications, 
        images, ratings, and all available prices from different retailers
    """
    params = {
        "product_id": product_id,
        "country": country,
        "language": language
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/product-details-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()


print("✅ Product Specialist tools defined:")
print("   • search_products - Search for products across retailers (API v2)")
print("   • get_product_details - Get detailed product specifications (API v2)")


✅ Product Specialist tools defined:
   • search_products - Search for products across retailers (API v2)
   • get_product_details - Get detailed product specifications (API v2)


In [3]:
# Test Product Specialist Tools
# Direct API calls to test the RapidAPI integration (v2 endpoints)

async def test_search_api(query: str, max_price: float = None, limit: int = 10) -> dict:
    """Direct API call for testing search-v2 endpoint."""
    params = {
        "q": query,
        "country": "us",
        "language": "en",
        "limit": str(limit),
        "page": "1",
        "sort_by": "BEST_MATCH",
        "product_condition": "ANY"
    }
    if max_price:
        params["max_price"] = str(max_price)
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/search-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()

async def test_details_api(product_id: str) -> dict:
    """Direct API call for testing product-details-v2 endpoint."""
    params = {
        "product_id": product_id,
        "country": "us",
        "language": "en"
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/product-details-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()

async def test_product_tools():
    """Test the product search and details APIs."""
    
    print("=" * 60)
    print("Testing search_products API (search-v2)")
    print("=" * 60)
    
    # Test search - using same query as successful curl request
    search_result = await test_search_api(
        query="Nike shoes",
        limit=10
    )
    
    print(f"\n🔍 Search: 'Nike shoes' (limit 10)")
    print(f"Status: {'✅ Success' if search_result.get('status') == 'OK' else '❌ Failed'}")
    
    # Parse response: data -> products (array)
    products = search_result.get("data", {}).get("products", [])
    print(f"Products found: {len(products)}")
    
    product_id = None
    if products:
        print(f"\nTop {min(5, len(products))} Results:")
        print("-" * 55)
        for i, product in enumerate(products[:5], 1):
            # Parse product fields based on actual API response structure
            title = product.get("product_title", "N/A")[:45]
            rating = product.get("product_rating")  # number like 4.5
            num_reviews = product.get("product_num_reviews")  # number like 10106
            num_offers = product.get("product_num_offers")  # number like 4982
            typical_price = product.get("typical_price_range")  # array like ["$68", "$115"] or null
            
            # offer is a single object (not array) containing the headline offer
            # Use "or {}" because offer can be null in the response
            offer = product.get("offer") or {}
            offer_price = offer.get("price", "N/A")  # string like "$115.00"
            store_name = offer.get("store_name", "N/A")
            on_sale = offer.get("on_sale", False)
            original_price = offer.get("original_price")
            
            print(f"  {i}. {title}...")
            print(f"     💰 Price: {offer_price} at {store_name}", end="")
            if on_sale and original_price:
                print(f" (was {original_price})")
            else:
                print()
            if rating and num_reviews:
                print(f"     ⭐ Rating: {rating}/5 ({num_reviews:,} reviews)")
            elif rating:
                print(f"     ⭐ Rating: {rating}/5")
            else:
                print(f"     ⭐ Rating: N/A")
            if typical_price and len(typical_price) >= 2:
                print(f"     📊 Typical range: {typical_price[0]} - {typical_price[1]}")
            print()
            
            # Save first product_id for details test
            if i == 1:
                product_id = product.get("product_id")
    
    print("=" * 60)
    print("Testing get_product_details API (product-details-v2)")
    print("=" * 60)
    
    if product_id:
        # Truncate product_id for display (it's very long)
        display_id = product_id[:50] + "..." if len(product_id) > 50 else product_id
        print(f"\n📦 Product ID: {display_id}")
        
        details_result = await test_details_api(product_id=product_id)
        
        print(f"Status: {'✅ Success' if details_result.get('status') == 'OK' else '❌ Failed'}")
        
        # Parse product details response
        product_data = details_result.get("data", {})
        if product_data:
            title = product_data.get("product_title", "N/A")
            print(f"\n🏷️  Product: {title[:60]}...")
            print(f"⭐ Rating: {product_data.get('product_rating', 'N/A')}")
            print(f"📝 Reviews: {product_data.get('product_num_reviews', 'N/A')}")
            
            # Check product_attributes for specs (use "or {}" in case value is null)
            attrs = product_data.get("product_attributes") or {}
            if attrs:
                print(f"\n📋 Key Attributes:")
                for key in list(attrs.keys())[:5]:  # Show first 5 attributes
                    print(f"   • {key}: {attrs[key]}")
            
            # Check for offers - might be array in details or single offer
            # Use "or []" / "or {}" in case values are null
            offers = product_data.get("offers") or []
            offer = product_data.get("offer") or {}
            
            if offers and isinstance(offers, list):
                print(f"\n💳 Offers from {len(offers)} stores:")
                for off in offers[:3]:
                    store = off.get("store_name", "Unknown")
                    price = off.get("price", "N/A")
                    print(f"   • {store}: {price}")
            elif offer:
                print(f"\n💳 Headline Offer:")
                print(f"   • {offer.get('store_name', 'N/A')}: {offer.get('price', 'N/A')}")
        else:
            print("⚠️ No product data in response")
            print(f"Response keys: {list(details_result.keys())}")
    else:
        print("⚠️ No product_id available to test get_product_details")
    
    print("\n" + "=" * 60)
    print("✅ Tool tests completed")
    print("=" * 60)

# Run the test
await test_product_tools()


Testing search_products API (search-v2)

🔍 Search: 'Nike shoes' (limit 10)
Status: ✅ Success
Products found: 10

Top 5 Results:
-------------------------------------------------------
  1. Nike Men's Revolution 8 Running Shoes...
     💰 Price: $56.97 at Nike (was $75.00)
     ⭐ Rating: 4.6/5 (1,625 reviews)
     📊 Typical range: $56 - $75

  2. Nike Women's Ebernon Low...
     💰 Price: $70.00 at Nike
     ⭐ Rating: 4.4/5 (239 reviews)
     📊 Typical range: $49 - $76

  3. ...
     💰 Price: N/A at N/A
     ⭐ Rating: N/A

  4. Nike Kids' Court Borough Low Recraft...
     💰 Price: $31.42 at DICK'S Sporting Goods (was $66.99)
     ⭐ Rating: 4.6/5 (3,069 reviews)
     📊 Typical range: $31 - $69

  5. ...
     💰 Price: N/A at N/A
     ⭐ Rating: N/A

Testing get_product_details API (product-details-v2)

📦 Product ID: catalogid:602782393795019172,productid:,gpcid:5477...
Status: ✅ Success

🏷️  Product: Nike Men's Revolution 8 Running Shoes...
⭐ Rating: 4.6
📝 Reviews: 1625

💳 Headline Offer:
  

In [4]:
# Review Analyst Tools
# Tools for fetching and analyzing product reviews

from typing import Optional, List
from collections import Counter
import re

# Valid sort_by options for the reviews API v2
VALID_REVIEWS_SORT_OPTIONS = {"MOST_RELEVANT", "DATE"}


@function_tool
async def get_product_reviews(
    product_id: str,
    country: str = "us",
    language: str = "en",
    limit: int = 10,
    sort_by: str = "MOST_RELEVANT"
) -> dict:
    """
    Fetch customer reviews and ratings for a specific product using API v2.
    
    IMPORTANT: The product_id MUST be the exact ID returned from search_products results.
    It looks like: "catalogid:123456,productid:,gpcid:789..." 
    Do NOT make up product IDs like "sony_wh_1000xm4" - these will return empty results.
    
    Args:
        product_id: Product ID from search_products results (long string starting with "catalogid:")
        country: Country code for localized reviews (default: us)
        language: Language code (default: en)
        limit: Number of reviews to fetch (default: 10)
        sort_by: Sort order - MOST_RELEVANT or DATE (default: MOST_RELEVANT)
    
    Returns:
        Dictionary containing product reviews, ratings, and review breakdown
    """
    # Validate and normalize sort_by parameter
    # Map common alternatives to valid API options
    sort_mapping = {
        "NEWEST": "DATE",
        "RECENT": "DATE", 
        "DATE_DESC": "DATE",
        "HIGHEST_RATING": "MOST_RELEVANT",
        "LOWEST_RATING": "MOST_RELEVANT",
        "BEST": "MOST_RELEVANT",
    }
    normalized_sort = sort_mapping.get(sort_by.upper(), sort_by.upper())
    if normalized_sort not in VALID_REVIEWS_SORT_OPTIONS:
        normalized_sort = "MOST_RELEVANT"  # Default fallback
    
    params = {
        "product_id": product_id,
        "country": country,
        "language": language,
        "limit": str(limit),
        "sort_by": normalized_sort
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/product-reviews-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()


# Sentiment analysis implementation (separated for testability)
def _analyze_sentiment_impl(
    reviews: List[dict],
    focus_area: Optional[str] = None
) -> dict:
    """
    Implementation of sentiment analysis logic.
    Separated from the @function_tool decorator for direct testing.
    """
    if not reviews:
        return {
            "overall_sentiment": "neutral",
            "sentiment_score": 0.0,
            "total_analyzed": 0,
            "pros": [],
            "cons": [],
            "common_themes": [],
            "notable_reviews": [],
            "red_flags": [],
            "recommendation": "No reviews available to analyze."
        }
    
    # Positive and negative keyword patterns
    positive_keywords = [
        "great", "excellent", "amazing", "love", "perfect", "best", "awesome",
        "fantastic", "wonderful", "quality", "comfortable", "recommend", "worth",
        "happy", "satisfied", "impressive", "solid", "reliable", "easy"
    ]
    negative_keywords = [
        "poor", "bad", "terrible", "worst", "broken", "disappointed", "waste",
        "cheap", "defective", "problem", "issue", "fail", "return", "refund",
        "uncomfortable", "unreliable", "slow", "loud", "flimsy", "avoid"
    ]
    
    # Categorize reviews by rating
    positive_reviews = []  # 4-5 stars
    neutral_reviews = []   # 3 stars
    negative_reviews = []  # 1-2 stars
    
    all_text = []
    ratings = []
    
    for review in reviews:
        rating = review.get("rating") or review.get("review_rating")
        content = review.get("review_text") or review.get("content") or review.get("review_content") or ""
        title = review.get("review_title") or review.get("title") or ""
        
        full_text = f"{title} {content}".lower()
        all_text.append(full_text)
        
        if rating:
            ratings.append(rating)
            if rating >= 4:
                positive_reviews.append(review)
            elif rating == 3:
                neutral_reviews.append(review)
            else:
                negative_reviews.append(review)
    
    # Calculate sentiment score
    if ratings:
        avg_rating = sum(ratings) / len(ratings)
        sentiment_score = (avg_rating - 3) / 2  # Convert 1-5 scale to -1 to 1
    else:
        sentiment_score = 0.0
    
    # Determine overall sentiment
    if sentiment_score > 0.3:
        overall_sentiment = "positive"
    elif sentiment_score < -0.3:
        overall_sentiment = "negative"
    elif len(positive_reviews) > 0 and len(negative_reviews) > 0:
        overall_sentiment = "mixed"
    else:
        overall_sentiment = "neutral"
    
    # Extract pros and cons from review text
    combined_text = " ".join(all_text)
    
    pros = []
    cons = []
    
    for keyword in positive_keywords:
        if keyword in combined_text:
            # Count occurrences
            count = combined_text.count(keyword)
            if count >= 2:  # Only include if mentioned multiple times
                pros.append(keyword)
    
    for keyword in negative_keywords:
        if keyword in combined_text:
            count = combined_text.count(keyword)
            if count >= 2:
                cons.append(keyword)
    
    # Filter by focus area if specified
    if focus_area:
        focus_lower = focus_area.lower()
        relevant_text = [t for t in all_text if focus_lower in t]
        if relevant_text:
            combined_text = " ".join(relevant_text)
    
    # Identify common themes (simple word frequency)
    words = re.findall(r'\b[a-z]{4,}\b', combined_text)
    word_counts = Counter(words)
    # Remove common stop words
    stop_words = {"this", "that", "with", "have", "from", "they", "been", "were", "will", "would", "could", "should", "about", "into", "than", "them", "then", "these", "very", "just", "your", "some", "more", "also", "what", "when"}
    common_themes = [word for word, count in word_counts.most_common(20) 
                     if word not in stop_words and count >= 3][:10]
    
    # Identify red flags (serious concerns)
    red_flag_patterns = ["defective", "broke", "broken", "dangerous", "safety", "recall", "fire", "burn", "shock", "injury"]
    red_flags = [pattern for pattern in red_flag_patterns if pattern in combined_text]
    
    # Find notable reviews (high helpful votes or detailed)
    notable_reviews = []
    for review in reviews[:5]:  # Check first 5 reviews
        helpful = review.get("review_helpful_count") or review.get("helpful_votes") or 0
        content = review.get("review_text") or review.get("content") or ""
        if helpful > 10 or len(content) > 200:
            notable_reviews.append({
                "title": review.get("review_title") or review.get("title") or "No title",
                "rating": review.get("rating") or review.get("review_rating"),
                "helpful_votes": helpful,
                "snippet": content[:150] + "..." if len(content) > 150 else content
            })
    
    # Generate recommendation
    if overall_sentiment == "positive":
        recommendation = f"Generally well-reviewed with {len(positive_reviews)} positive reviews. " \
                        f"Most customers are satisfied with this product."
    elif overall_sentiment == "negative":
        recommendation = f"Caution advised - {len(negative_reviews)} negative reviews. " \
                        f"Common complaints include: {', '.join(cons[:3]) if cons else 'various issues'}."
    elif overall_sentiment == "mixed":
        recommendation = f"Mixed reviews ({len(positive_reviews)} positive, {len(negative_reviews)} negative). " \
                        f"Consider both pros and cons carefully before purchasing."
    else:
        recommendation = "Insufficient data to make a strong recommendation."
    
    return {
        "overall_sentiment": overall_sentiment,
        "sentiment_score": round(sentiment_score, 2),
        "total_analyzed": len(reviews),
        "rating_breakdown": {
            "positive": len(positive_reviews),
            "neutral": len(neutral_reviews),
            "negative": len(negative_reviews)
        },
        "average_rating": round(avg_rating, 1) if ratings else None,
        "pros": pros[:5],
        "cons": cons[:5],
        "common_themes": common_themes,
        "notable_reviews": notable_reviews[:3],
        "red_flags": red_flags,
        "recommendation": recommendation
    }


# Wrap the implementation with @function_tool for agent use
@function_tool(strict_mode=False)
def analyze_sentiment(
    reviews: Optional[List[dict]] = None,
    focus_area: Optional[str] = None
) -> dict:
    """
    Analyze review text to extract sentiment, themes, pros, and cons.
    This is a local processing tool - no external API call.
    
    Args:
        reviews: Array of review objects from get_product_reviews (default: empty list)
        focus_area: Optional specific aspect to analyze (e.g., "battery", "durability")
    
    Returns:
        Dictionary containing sentiment analysis results including:
        - overall_sentiment: positive, negative, mixed, or neutral
        - sentiment_score: -1.0 to 1.0
        - pros: commonly mentioned positive aspects
        - cons: commonly mentioned negative aspects
        - common_themes: frequently mentioned topics
        - red_flags: serious concerns mentioned by multiple reviewers
    """
    # Handle None or empty reviews gracefully
    if reviews is None:
        reviews = []
    return _analyze_sentiment_impl(reviews, focus_area)


print("✅ Review Analyst tools defined:")
print("   • get_product_reviews - Fetch reviews from RapidAPI (API v2)")
print("   • analyze_sentiment - Local sentiment analysis")


✅ Review Analyst tools defined:
   • get_product_reviews - Fetch reviews from RapidAPI (API v2)
   • analyze_sentiment - Local sentiment analysis


In [5]:
# Test Review Analyst Tools
# Direct API calls to test the review fetching and sentiment analysis (v2 endpoints)

async def test_reviews_api(product_id: str, limit: int = 10, sort_by: str = "MOST_RELEVANT") -> dict:
    """Direct API call for testing product-reviews-v2 endpoint."""
    params = {
        "product_id": product_id,
        "country": "us",
        "language": "en",
        "limit": str(limit),
        "sort_by": sort_by
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{RAPIDAPI_BASE_URL}/product-reviews-v2",
            headers=get_rapidapi_headers(),
            params=params,
            timeout=30.0
        )
        response.raise_for_status()
        return response.json()


async def test_review_analyst_tools():
    """Test the review fetching and sentiment analysis tools."""
    
    print("=" * 60)
    print("Testing Review Analyst Tools")
    print("=" * 60)
    
    # Step 1: First get a product_id by searching
    print("\n📦 Step 1: Getting a product_id from search...")
    
    search_params = {
        "q": "Nike shoes",
        "country": "us",
        "language": "en",
        "limit": "1",
        "page": "1",
        "sort_by": "BEST_MATCH",
        "product_condition": "ANY"
    }
    
    async with httpx.AsyncClient() as client:
        search_response = await client.get(
            f"{RAPIDAPI_BASE_URL}/search-v2",
            headers=get_rapidapi_headers(),
            params=search_params,
            timeout=30.0
        )
        search_response.raise_for_status()
        search_result = search_response.json()
    
    products = search_result.get("data", {}).get("products", [])
    if not products:
        print("❌ No products found to test reviews")
        return
    
    product = products[0]
    product_id = product.get("product_id")
    product_title = product.get("product_title", "Unknown")[:50]
    
    print(f"✅ Found product: {product_title}...")
    print(f"   Product ID: {product_id[:50]}...")
    
    # Step 2: Fetch reviews for this product
    print("\n" + "=" * 60)
    print("📝 Step 2: Testing get_product_reviews API (product-reviews-v2)")
    print("=" * 60)
    
    try:
        reviews_result = await test_reviews_api(product_id=product_id, limit=10)
        
        print(f"\nStatus: {'✅ Success' if reviews_result.get('status') == 'OK' else '❌ Failed'}")
        
        # Parse the reviews response
        data = reviews_result.get("data", {})
        reviews = data.get("reviews", [])
        
        # Try different possible field names for rating info
        avg_rating = data.get("product_rating") or data.get("average_rating") or data.get("rating")
        total_reviews = data.get("product_num_reviews") or data.get("total_reviews") or len(reviews)
        
        print(f"\n📊 Review Summary:")
        print(f"   Average Rating: {avg_rating or 'N/A'}")
        print(f"   Total Reviews Available: {total_reviews}")
        print(f"   Reviews Fetched: {len(reviews)}")
        
        # Show rating breakdown if available
        rating_breakdown = data.get("rating_breakdown") or data.get("rating_distribution")
        if rating_breakdown:
            print(f"\n⭐ Rating Breakdown:")
            for key, value in rating_breakdown.items():
                print(f"   {key}: {value}")
        
        # Show sample reviews
        if reviews:
            print(f"\n📖 Sample Reviews ({min(3, len(reviews))}):")
            print("-" * 55)
            for i, review in enumerate(reviews[:3], 1):
                # Handle different possible field names
                author = review.get("review_author") or review.get("author") or "Anonymous"
                rating = review.get("review_rating") or review.get("rating") or "N/A"
                title = review.get("review_title") or review.get("title") or "No title"
                content = review.get("review_text") or review.get("content") or review.get("review_content") or ""
                source = review.get("source") or review.get("review_source") or "Unknown"
                
                print(f"\n  Review {i}:")
                print(f"  👤 Author: {author}")
                print(f"  ⭐ Rating: {rating}/5")
                print(f"  📌 Title: {title[:50]}..." if len(str(title)) > 50 else f"  📌 Title: {title}")
                print(f"  📝 Content: {content[:100]}..." if len(content) > 100 else f"  📝 Content: {content}")
                print(f"  🏪 Source: {source}")
        
        # Step 3: Test sentiment analysis
        print("\n" + "=" * 60)
        print("🧠 Step 3: Testing analyze_sentiment (Local Processing)")
        print("=" * 60)
        
        if reviews:
            # Call the implementation function directly for testing
            # (The @function_tool decorated version is for agent use)
            sentiment_result = _analyze_sentiment_impl(reviews=reviews, focus_area=None)
            
            print(f"\n📊 Sentiment Analysis Results:")
            print(f"   Overall Sentiment: {sentiment_result.get('overall_sentiment', 'N/A').upper()}")
            print(f"   Sentiment Score: {sentiment_result.get('sentiment_score', 'N/A')} (-1.0 to 1.0)")
            print(f"   Reviews Analyzed: {sentiment_result.get('total_analyzed', 0)}")
            
            rating_breakdown = sentiment_result.get("rating_breakdown", {})
            if rating_breakdown:
                print(f"\n   Rating Breakdown:")
                print(f"     👍 Positive (4-5⭐): {rating_breakdown.get('positive', 0)}")
                print(f"     😐 Neutral (3⭐): {rating_breakdown.get('neutral', 0)}")
                print(f"     👎 Negative (1-2⭐): {rating_breakdown.get('negative', 0)}")
            
            pros = sentiment_result.get("pros", [])
            cons = sentiment_result.get("cons", [])
            
            if pros:
                print(f"\n   ✅ Pros: {', '.join(pros)}")
            if cons:
                print(f"   ❌ Cons: {', '.join(cons)}")
            
            themes = sentiment_result.get("common_themes", [])
            if themes:
                print(f"\n   🏷️ Common Themes: {', '.join(themes[:5])}")
            
            red_flags = sentiment_result.get("red_flags", [])
            if red_flags:
                print(f"\n   🚩 Red Flags: {', '.join(red_flags)}")
            
            print(f"\n   💡 Recommendation: {sentiment_result.get('recommendation', 'N/A')}")
        else:
            print("\n⚠️ No reviews available to analyze sentiment")
    
    except httpx.HTTPStatusError as e:
        print(f"\n❌ API Error: {e.response.status_code}")
        print(f"   URL: {e.request.url}")
        print(f"   Response: {e.response.text[:200]}...")
    except Exception as e:
        print(f"\n❌ Error: {type(e).__name__}: {str(e)}")
    
    print("\n" + "=" * 60)
    print("✅ Review Analyst tool tests completed")
    print("=" * 60)

# Run the test
await test_review_analyst_tools()


Testing Review Analyst Tools

📦 Step 1: Getting a product_id from search...
✅ Found product: Nike Men's Air Force 1 07...
   Product ID: catalogid:17683191182260831047,productid:,gpcid:16...

📝 Step 2: Testing get_product_reviews API (product-reviews-v2)

Status: ✅ Success

📊 Review Summary:
   Average Rating: 4.7
   Total Reviews Available: 18704
   Reviews Fetched: 10

📖 Sample Reviews (3):
-------------------------------------------------------

  Review 1:
  👤 Author: JeffH
  ⭐ Rating: 5/5
  📌 Title: I bought this product AGAIN
  📝 Content: The first pair was so wonderful that I bought a second pair 10 years later. The sole wore out due to...
  🏪 Source: finishline.com

  Review 2:
  👤 Author: Anonymous
  ⭐ Rating: 4/5
  📌 Title: No title
  📝 Content: Delivery was good no issues with that as it was delivered with intact and original box.Shoes quality...
  🏪 Source: myntra.com

  Review 3:
  👤 Author: Mom of four
  ⭐ Rating: 5/5
  📌 Title: Dress up casual
  📝 Content: My son looks f

In [6]:
# Smart Shopping Assistant - Agent Definitions
# Three agents with bidirectional handoffs as described in the design

from agents import Agent, Runner, RunConfig, OpenAIProvider

# =============================================================================
# Provider Configuration
# =============================================================================

# IMPORTANT: Bidirectional handoffs require ALL agents to use OpenAI models!
#
# Why? The OpenAI Agents SDK's native handoff mechanism uses the Responses API.
# When an Anthropic agent (using Chat Completions) hands back to OpenAI, the
# conversation history contains "__fake_id__" placeholders that OpenAI rejects.
#
# One-way handoffs (OpenAI → Anthropic) work, but returning (Anthropic → OpenAI) fails.
#
# For multi-provider without bidirectional handoffs, use manual orchestration
# like the email_battle notebook (separate Runner.run calls for each agent).

openai_provider = OpenAIProvider(api_key=OPENAI_API_KEY)

# =============================================================================
# Model Selection
# =============================================================================

# All agents use OpenAI models to enable bidirectional native handoffs
# - Concierge: gpt-5.2-pro for advanced orchestration and reasoning
# - Specialists: gpt-5.2 for efficient tool use with lower latency/cost
CONCIERGE_MODEL = "gpt-5.2-pro"  # Advanced reasoning for orchestration
SPECIALIST_MODEL = "gpt-5.2"    # Standard model for tool-based tasks

print(f"📋 Model Configuration:")
print(f"   Concierge: {CONCIERGE_MODEL} (orchestration + reasoning)")
print(f"   Specialists: {SPECIALIST_MODEL} (efficient tool use)")
print()
print("ℹ️  All agents use OpenAI models for bidirectional handoff compatibility")

# =============================================================================
# System Prompts
# =============================================================================

CONCIERGE_INSTRUCTIONS = """
You are the Concierge Agent for a Smart Shopping Assistant. You are the main 
orchestrator that users interact with directly.

## Your Role

You help users find, compare, and research products. You do NOT perform the actual 
product searches or review analysis yourself. Instead, you delegate to specialized 
agents and then present their findings to users in a friendly, helpful manner.

## Your Specialist Agents

You have two specialist agents you can hand off to:

1. **Product Specialist** - For product searches, price comparisons, and specifications
   - Use when users want to find products, compare prices, or get product details
   - Examples: "Find me a laptop under $1000", "Compare AirPods prices", "What are the specs?"

2. **Review Analyst** - For review analysis and sentiment insights
   - Use when users want to know about product reviews, ratings, or customer feedback
   - Examples: "What do reviews say about this?", "Is this product reliable?", "Any complaints?"

## How to Hand Off

When you identify the user's intent:
1. Briefly acknowledge their request
2. Transfer to the appropriate specialist using the transfer function
3. After receiving results from the specialist, synthesize and present to the user

## When Specialists Hand Back to You

After a specialist completes their task, they will transfer back to you with their 
findings. Your job is then to:
1. Understand the specialist's results
2. Present the information in a user-friendly way
3. Ask if the user needs anything else

## Guidelines

- Be warm, helpful, and conversational
- Don't try to search or analyze yourself - always delegate to specialists
- If a request involves both products AND reviews, handle one at a time
- Keep your responses concise when handing off, detailed when presenting results
- If the user's request is unclear, ask clarifying questions before delegating
"""

PRODUCT_SPECIALIST_INSTRUCTIONS = """
You are the Product Specialist Agent for a Smart Shopping Assistant. You help users 
find and compare products using your search and details tools.

## Your Role

You receive requests from the Concierge Agent to find products, compare prices, or get 
specifications. After completing your task, you MUST transfer back to the Concierge 
with your findings.

## Your Tools

### search_products
Use this tool to find products matching user criteria.
- Always include relevant keywords from the user's request in the query
- Apply price filters when the user specifies a budget
- Use sort_by to match user preferences:
  - "cheapest" → sort_by: "LOWEST_PRICE"
  - "best rated" → sort_by: "TOP_RATED" 
  - "most relevant" → sort_by: "BEST_MATCH"
- Default to product_condition: "ANY" unless user asks for new/used/refurbished

### get_product_details
Use this tool when you need:
- Full technical specifications
- Complete price comparison across all stores
- Detailed product information

Only use this after search_products when you have a product_id, or when the user 
asks about a specific product's specs.

## Workflow

1. RECEIVE the request from Concierge (user wants products, prices, or specs)
2. ANALYZE what tools you need:
   - Product search → use search_products
   - Need specs for specific product → use get_product_details
   - Price comparison → use search_products (offers are included)
3. EXECUTE the appropriate tool(s)
4. SUMMARIZE the results clearly:
   - For search: List top products with key info (title, price, rating, stores)
   - For specs: Present specifications in an organized format
   - For prices: Show the price comparison across stores
5. TRANSFER back to Concierge using transfer_to_concierge

## Guidelines

### Search Queries
- Extract the main product type: "gaming laptop with RTX 4060" → query: "gaming laptop RTX 4060"
- Include key features mentioned: "16GB RAM", "15 inch screen", etc.
- Don't over-specify: let the API find relevant matches

### Price Handling
- "under $1500" → max_price: 1500
- "at least $500" → min_price: 500
- "$500-$1000" → min_price: 500, max_price: 1000
- "around $800" → min_price: 700, max_price: 900

### Presenting Results
- Always mention the number of results found
- Highlight the best value option (good rating + competitive price)
- Note price differences between stores when significant
- Include ratings and review counts for credibility

## Important

- ALWAYS transfer back to Concierge after completing your task
- Do NOT respond directly to the user - let Concierge handle user communication
- Include enough context in your summary for Concierge to present to the user
"""

REVIEW_ANALYST_INSTRUCTIONS = """
You are the Review Analyst Agent for a Smart Shopping Assistant. You help users 
understand product reviews and customer sentiment.

## Your Role

You receive requests from the Concierge Agent to analyze reviews for specific products.
You MUST use your tools to fetch and analyze reviews before transferring back.

## Your Tools

### search_products
Use this tool FIRST if you don't have a valid product_id.
- Search by product name to find products and get their product_id
- Returns products with product_id values you can use for get_product_reviews
- Example: search_products(query="Sony WH-1000XM4", limit=1)

### get_product_reviews
Use this tool to fetch customer reviews for a product.
- Requires product_id from search_products results (long string like "catalogid:...")
- Valid sort_by options: MOST_RELEVANT or DATE

### analyze_sentiment  
Use this tool to analyze review data and extract insights.
- Takes the reviews array from get_product_reviews
- Can focus on specific aspects (e.g., "battery", "durability") with focus_area
- Returns sentiment score, pros, cons, themes, and red flags

## Workflow - ALWAYS USE YOUR TOOLS

1. RECEIVE the request from Concierge (user wants review analysis)
2. CHECK for product_id in conversation context:
   - If you have a product_id (long string like "catalogid:..."), skip to step 4
   - If you only have a product name, proceed to step 3
3. SEARCH for the product:
   - Call search_products with the product name
   - Extract the product_id from the first result
4. FETCH reviews:
   - Call get_product_reviews with the product_id
5. ANALYZE reviews:
   - If reviews are returned, call analyze_sentiment
6. SUMMARIZE your findings:
   - Overall sentiment (positive/negative/mixed)
   - Key pros and cons
   - Common themes or concerns
   - Red flags if any
   - Your recommendation
7. TRANSFER back to Concierge with your findings

## IMPORTANT - Always Try Your Tools!

- If you only have a product NAME, use search_products first to get product_id
- ALWAYS call get_product_reviews before handing back
- Never hand back to Concierge without having attempted to use your tools

## Guidelines

### Presenting Analysis
- Lead with the overall sentiment and score
- List top 3 pros and top 3 cons  
- Highlight any red flags prominently (safety issues, defects, etc.)
- Include notable quotes from helpful reviews
- End with a clear recommendation

### If No Reviews Found
If get_product_reviews returns empty:
- Report this clearly: "I searched for [product] and found it, but no reviews are available"
- Still transfer back to Concierge with this information

## Important

- ALWAYS use your tools before transferring back
- Use search_products if you need to find a product_id
- Be objective in your analysis - present both positive and negative findings
"""

# =============================================================================
# Agent Definitions
# =============================================================================

# Note: We create agents in a specific order because of handoff dependencies
# Specialists need to hand back to Concierge, and Concierge needs to hand to specialists

# Step 1: Create Product Specialist (will add handoffs later)
product_specialist = Agent(
    name="Product Specialist",
    instructions=PRODUCT_SPECIALIST_INSTRUCTIONS,
    model=SPECIALIST_MODEL,
    tools=[search_products, get_product_details],
    # handoffs will be set after concierge is created
)

# Step 2: Create Review Analyst (will add handoffs later)
# Note: Review Analyst has search_products to find product_id before fetching reviews
review_analyst = Agent(
    name="Review Analyst", 
    instructions=REVIEW_ANALYST_INSTRUCTIONS,
    model=SPECIALIST_MODEL,
    tools=[search_products, get_product_reviews, analyze_sentiment],
    # handoffs will be set after concierge is created
)

# Step 3: Create Concierge with handoffs to specialists
concierge = Agent(
    name="Concierge",
    instructions=CONCIERGE_INSTRUCTIONS,
    model=CONCIERGE_MODEL,
    handoffs=[product_specialist, review_analyst],  # Can delegate to specialists
)

# Step 4: Add bidirectional handoffs - specialists can hand back to Concierge
product_specialist.handoffs = [concierge]
review_analyst.handoffs = [concierge]

# =============================================================================
# Run Configuration
# =============================================================================

# Single OpenAI provider for all agents - required for bidirectional handoffs
run_config = RunConfig(model_provider=openai_provider)

# =============================================================================
# Verification
# =============================================================================

print()
print("✅ Smart Shopping Assistant Agents Defined")
print()
print("🎯 CONCIERGE AGENT")
print(f"   Model: {CONCIERGE_MODEL}")
print(f"   Can hand off to: {[a.name for a in concierge.handoffs]}")
print(f"   Tools: None (orchestrator only)")
print()
print("🔍 PRODUCT SPECIALIST AGENT")
print(f"   Model: {SPECIALIST_MODEL}")
print(f"   Can hand off to: {[a.name for a in product_specialist.handoffs]}")
print(f"   Tools: {[t.name for t in product_specialist.tools]}")
print()
print("⭐ REVIEW ANALYST AGENT")
print(f"   Model: {SPECIALIST_MODEL}")
print(f"   Can hand off to: {[a.name for a in review_analyst.handoffs]}")
print(f"   Tools: {[t.name for t in review_analyst.tools]}")
print()
print("✨ Bidirectional Handoffs Ready!")
print("   All agents use OpenAI models (required for native SDK handoffs)")
print("   Concierge ↔ Product Specialist ↔ Concierge")
print("   Concierge ↔ Review Analyst ↔ Concierge")


📋 Model Configuration:
   Concierge: gpt-5.2-pro (orchestration + reasoning)
   Specialists: gpt-5.2 (efficient tool use)

ℹ️  All agents use OpenAI models for bidirectional handoff compatibility

✅ Smart Shopping Assistant Agents Defined

🎯 CONCIERGE AGENT
   Model: gpt-5.2-pro
   Can hand off to: ['Product Specialist', 'Review Analyst']
   Tools: None (orchestrator only)

🔍 PRODUCT SPECIALIST AGENT
   Model: gpt-5.2
   Can hand off to: ['Concierge']
   Tools: ['search_products', 'get_product_details']

⭐ REVIEW ANALYST AGENT
   Model: gpt-5.2
   Can hand off to: ['Concierge']
   Tools: ['search_products', 'get_product_reviews', 'analyze_sentiment']

✨ Bidirectional Handoffs Ready!
   All agents use OpenAI models (required for native SDK handoffs)
   Concierge ↔ Product Specialist ↔ Concierge
   Concierge ↔ Review Analyst ↔ Concierge


In [7]:
# Run Smart Shopping Assistant with Tracing
# This cell runs an interactive session with the Concierge agent

from agents import trace, Runner
from IPython.display import Markdown, display

# Note: run_config is already defined in the Agent Definitions cell above
# All agents use OpenAI models with a single provider for native handoffs


# =============================================================================
# Run Function with Tracing
# =============================================================================

async def run_shopping_assistant(user_query: str, trace_name: str = "Smart Shopping Assistant"):
    """
    Run the Smart Shopping Assistant with a user query.
    
    Args:
        user_query: The user's question or request
        trace_name: Name for the trace (viewable in OpenAI dashboard)
    
    Returns:
        The final output from the assistant
    """
    with trace(trace_name):
        result = await Runner.run(
            starting_agent=concierge,
            input=user_query,
            run_config=run_config
        )
        return result


async def chat_session(queries: list[str], session_name: str = "Shopping Session"):
    """
    Run a multi-turn chat session with the Smart Shopping Assistant.
    
    Args:
        queries: List of user queries to process in sequence
        session_name: Name for the trace session
    """
    print("=" * 70)
    print(f"🛒 SMART SHOPPING ASSISTANT - {session_name}")
    print("=" * 70)
    
    for i, query in enumerate(queries, 1):
        print(f"\n{'─' * 70}")
        print(f"👤 USER ({i}/{len(queries)}):")
        print(f"   {query}")
        print(f"{'─' * 70}")
        
        result = await run_shopping_assistant(
            user_query=query,
            trace_name=f"{session_name} - Query {i}"
        )
        
        print(f"\n🤖 ASSISTANT:")
        print(f"{result.final_output}")
        
        # Show which agent produced the final response
        if hasattr(result, 'last_agent') and result.last_agent:
            print(f"\n   [Final response from: {result.last_agent.name}]")
    
    print(f"\n{'=' * 70}")
    print("✅ Session complete! View traces at: https://platform.openai.com/traces")
    print("=" * 70)


# =============================================================================
# Example Queries
# =============================================================================

# Example queries to test different agent paths
example_queries = [
    # Product search (→ Product Specialist)
    "Find me a gaming laptop under $1500 with good reviews",
    
    # Review analysis (→ Review Analyst) - would need product_id from previous search
    # "What do customers say about the ASUS ROG laptop?",
    
    # Price comparison (→ Product Specialist)
    # "What's the cheapest place to buy AirPods Pro?",
]

print("✅ Shopping Assistant runner ready!")
print()
print("📝 Example usage:")
print('   result = await run_shopping_assistant("Find me wireless headphones under $100")')
print()
print("📝 Or run a multi-turn session:")
print('   await chat_session(["Find laptops", "What about reviews?"])')
print()
print("🔍 View traces at: https://platform.openai.com/traces")


✅ Shopping Assistant runner ready!

📝 Example usage:
   result = await run_shopping_assistant("Find me wireless headphones under $100")

📝 Or run a multi-turn session:
   await chat_session(["Find laptops", "What about reviews?"])

🔍 View traces at: https://platform.openai.com/traces


In [8]:
# Test Run: Single Query
# Run a single query through the Smart Shopping Assistant

# More specific query to avoid follow-up questions:
# - Specifies type (over-ear)
# - Includes budget ($100)
# - Mentions feature (noise cancellation)
# - Specifies quantity (top 5)
TEST_QUERY = "Search for the top 5 over-ear wireless headphones under $100 with noise cancellation"

result = await run_shopping_assistant(
    user_query=TEST_QUERY,
    trace_name="Test Query - Wireless Headphones"
)

print("=" * 70)
print("🛒 SMART SHOPPING ASSISTANT - Test Query")
print("=" * 70)
print()
print("👤 USER:")
print(f"   {TEST_QUERY}")
print()
print("─" * 70)
print()
print("🤖 ASSISTANT:")
print(result.final_output)
print()
print("─" * 70)

# Show execution details
if hasattr(result, 'last_agent') and result.last_agent:
    print(f"📍 Final agent: {result.last_agent.name}")

print()
print("🔍 View full trace at: https://platform.openai.com/traces")


[non-fatal] Tracing: server error 503, retrying.


🛒 SMART SHOPPING ASSISTANT - Test Query

👤 USER:
   Search for the top 5 over-ear wireless headphones under $100 with noise cancellation

──────────────────────────────────────────────────────────────────────

🤖 ASSISTANT:
## Findings for Concierge: Top over-ear wireless headphones **under $100** with **noise cancellation** (US)

I ran a **TOP_RATED** search with max price **$100**. Results skew heavily toward **Soundcore/Anker** budget ANC models (high ratings + lots of reviews). Below are the best 5 *actual over-ear ANC* options surfaced (excluding on-ear/non-ANC items like Sony WH-CH520 and excluding “noise-cancelling mic only” gaming headsets).

### Top picks (5)
1) **Anker Soundcore Q20i – Hybrid ANC (Over-ear)**
- **Price**: **$37.99** (Best Buy, new; “Best price” badge)
- **Rating**: **4.8/5** (**6,218** reviews) | **Offers**: 90
- **Highlights**: Hybrid ANC (4 mics), **40h with ANC / 60h normal**, app EQ, transparency, foldable, BT 5.0.

2) **Soundcore by Anker Life Q30 – Hybri

In [9]:
# Test Run: Multi-Agent Handoff
# This test triggers BOTH the Product Specialist AND Review Analyst
#
# Flow:
#   Query 1: "Search for Sony WH-1000XM4..." → Product Specialist (search_products)
#   Query 2: "What do the reviews say..." → Review Analyst (get_product_reviews + analyze_sentiment)

MULTI_AGENT_QUERIES = [
    # Query 1: Triggers Product Specialist
    # - Specific product name for reliable search
    # - Asks for top results with prices
    "Search for Sony WH-1000XM4 headphones and show me the top 3 results with prices",
    
    # Query 2: Triggers Review Analyst  
    # - References the product from previous search
    # - Asks for review analysis (sentiment, pros/cons)
    "What do the reviews say about the Sony WH-1000XM4? Give me the pros and cons."
]

# Run the multi-turn session
await chat_session(
    queries=MULTI_AGENT_QUERIES,
    session_name="Multi-Agent Demo (Product + Reviews)"
)


🛒 SMART SHOPPING ASSISTANT - Multi-Agent Demo (Product + Reviews)

──────────────────────────────────────────────────────────────────────
👤 USER (1/2):
   Search for Sony WH-1000XM4 headphones and show me the top 3 results with prices
──────────────────────────────────────────────────────────────────────

🤖 ASSISTANT:
Here are the top results I found for **Sony WH-1000XM4 headphones** (US), along with the prices shown:

1) **Sony WH-1000XM4 Wireless Noise-Canceling Over‑Ear Headphones (Black)** — **$370.49** (New)  
   - Store: **Best Buy (Marketplace)**  
   - Shipping: **$16.49** (delivery window Dec 30, 2025 – Jan 10, 2026)  
   - Link: https://www.bestbuy.com/product/sony-wh-1000xm4-wireless-noise-cancelling-over-the-ear-headphones-black/J7XSRH44JK/sku/10893014?ref=212&loc=marketplace

2) **Sony WH-1000XM4 Wireless Over‑Ear Bluetooth Headphones (Black)** — **$109.99** (Used)  
   - Store: **paymore.com**  
   - Shipping: **Free delivery** (Jan 2 – Jan 9) + **Free 30-day returns**  

## Test Results Summary

### Single Query Test 
**Query:** "Search for the top 5 over-ear wireless headphones under $100 with noise cancellation"

| Aspect | Result |
|--------|--------|
| **Path** | Concierge → Product Specialist |
| **Tools Used** | `search_products` (with max_price filter, TOP_RATED sort) |
| **Output** | 5 products with prices, ratings, store links |
| **Final Agent** | Product Specialist |

✅ Successfully filtered by price ($100 cap) and returned relevant ANC headphones.

---

### Multi-Agent Test 

**Query 1:** "Search for Sony WH-1000XM4 headphones and show me the top 3 results with prices"

| Aspect | Result |
|--------|--------|
| **Path** | Concierge → Product Specialist → Concierge |
| **Tools Used** | `search_products` |
| **Output** | 3 products with prices from different stores |

**Query 2:** "What do the reviews say about the Sony WH-1000XM4? Give me the pros and cons."

| Aspect | Result |
|--------|--------|
| **Path** | Concierge → Review Analyst → Concierge |
| **Tools Used** | `search_products` → `get_product_reviews` → `analyze_sentiment` |
| **Output** | Detailed pros/cons analysis based on ~31,979 reviews |

✅ **Both specialist agents successfully used their tools and handed back to Concierge.**

---

### Key Observations

1. **Bidirectional Handoffs Work**: All agents successfully transfer control and return results
2. **Tool Usage**: Both specialists properly invoke their tools before handing back
3. **Review Analyst Enhancement**: Adding `search_products` to Review Analyst allowed it to find the product_id independently
4. **Response Quality**: Both agents provide detailed, actionable information with prices, ratings, and specific recommendations
